<a href="https://colab.research.google.com/github/greatestgoat/kaggle-template/blob/PushNotebooks/project/Notebook/templateColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github.com/greatestgoat/kaggle-template/tree/PushNotebooks/project/Notebook/templateColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Configurations

In [2]:
!git clone -b PushNotebooks https://github.com/greatestgoat/kaggle-template.git

fatal: destination path 'kaggle-template' already exists and is not an empty directory.


In [3]:
%cd kaggle-template/

/content/kaggle-template


In [4]:
!pip install -qr ./requirements.txt

In [5]:
try:
    from google.colab import auth
    auth.authenticate_user()
except ImportError:
    pass

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
auth_path = '/content/drive/MyDrive/auth/template'

In [8]:
import os
from dotenv import load_dotenv
from pathlib import Path
env_path = auth_path + '/.env'
load_dotenv(dotenv_path=env_path)



GITHUB_OWNER = os.getenv('GITHUB_OWNER')
REPOSITORY_NAME = os.getenv("REPOSITORY_NAME")
PROJECT_ID = os.getenv("PROJECT_ID")
BUCKET_NAME = os.getenv("BUCKET_NAME")

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [9]:
BUCKET_NAME

'kaggleops-project'

In [10]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [11]:
cd ./project/src

/content/kaggle-template/project/src


In [12]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [13]:
blobs = storage_client.list_blobs(BUCKET_NAME)
for blob in blobs:
    filepath = blob.name
    if filepath[:7]=="mlruns/" and len(filepath)!=7:
        os.makedirs("/".join(filepath.split("/")[:-1]), exist_ok=True)
        blob.download_to_filename(filepath)

# Run script

In [14]:
!python helloworld.py

Hello world


# Save file to GCS(mlruns)

In [15]:
import glob

files = [f for f in glob.glob("mlruns/**", recursive=True) if f[:7]=="mlruns/" and len(f)!=7]
files_in_bucket = [f.name for f in storage_client.list_blobs(BUCKET_NAME)]
bucket = storage_client.get_bucket(BUCKET_NAME)

for f in files:
    try:
        if f not in files_in_bucket:
            blob = bucket.blob(f)
            blob.upload_from_filename(f)
    except:
        pass